# 🧠 13:00–15:00 응용 실습 — NIST PDF RAG (Chroma + LCEL)

이 노트북은 **`src/04_embeddings_chroma.py` + `src/05_rag_pipeline.py`** 내용을 하나로 합친 **실습 통합 버전**입니다.  
다음 순서로 진행합니다.

1) 환경 변수/경로 세팅  
2) PDF 로딩 → Document 목록 만들기  
3) Chunking(텍스트 분할)  
4) Embedding + Chroma 벡터스토어 인덱싱(persist)  
5) Retriever 생성  
6) LCEL 기반 RAG 체인 구축 (`question` → `answer`)  
7) 테스트 질의 & LangSmith Trace 확인 가이드  
8) 🧪 자유 실험 칸 (TODO 셀)

> ⚠️ 루트 경로에 `.env` 파일이 있고, `OPENAI_API_KEY`, `LANGCHAIN_API_KEY`, `LANGCHAIN_TRACING_V2=true`, `LANGCHAIN_PROJECT=gccare-rag-workshop` 가 설정되어 있어야 합니다.


## 1) 환경 변수 & 경로 세팅

In [3]:
from pathlib import Path
import os
from dotenv import load_dotenv

# 레포 루트 기준 .env
BASE_DIR = Path.cwd().resolve().parents[0]  # 노트북 위치에 따라 조정하세요
ENV_PATH = BASE_DIR / ".env"
load_dotenv(ENV_PATH)

print("BASE_DIR:", BASE_DIR)
print("OPENAI_API_KEY set?:", bool(os.getenv("OPENAI_API_KEY")))
print("LANGCHAIN_API_KEY set?:", bool(os.getenv("LANGCHAIN_API_KEY")))
print("LANGCHAIN_TRACING_V2:", os.getenv("LANGCHAIN_TRACING_V2"))
print("LANGCHAIN_PROJECT:", os.getenv("LANGCHAIN_PROJECT"))

DOCS_DIR = BASE_DIR / "data" / "docs"
CHROMA_DIR = BASE_DIR / "chroma_store"
print("DOCS_DIR exists?:", DOCS_DIR.exists())
CHROMA_DIR.mkdir(exist_ok=True)


BASE_DIR: /Users/sjcha/Documents/6. 개인작업 관리(smilechacha)/@모두의연구소/@모두의연구소 - 04 - GC케어&유비케어/202511_gccare/gccare-langchain-rag-ai
OPENAI_API_KEY set?: True
LANGCHAIN_API_KEY set?: True
LANGCHAIN_TRACING_V2: true
LANGCHAIN_PROJECT: gccare-rag-workshop
DOCS_DIR exists?: True


## 2) PDF 로딩 → Document 만들기

- `langchain_community.document_loaders.PyPDFLoader` 사용
- 각 페이지를 `Document`로 만들고, `metadata['source']`에 파일명 기록


In [5]:
from typing import List
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document

def load_all_pdfs(docs_dir: Path) -> List[Document]:
    pdf_paths = sorted(docs_dir.glob("*.pdf"))
    docs: List[Document] = []
    for path in pdf_paths:
        loader = PyPDFLoader(str(path))
        loaded = loader.load()
        for d in loaded:
            d.metadata.setdefault("source", path.name)
        docs.extend(loaded)
    return docs

docs = load_all_pdfs(DOCS_DIR)
print(f"Loaded pages: {len(docs)}")
if docs:
    print(docs[0].metadata)
    print(docs[0].page_content[:300])


Loaded pages: 139
{'producer': 'pdfTeX-1.40.24', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-01-24T14:45:46-05:00', 'author': 'National Institute of Standards and Technology', 'keywords': 'Artificial Intelligence (AI); AI; AI RMF; AI RMF 1.0; AI systems; trustworthy and responsible AI.', 'moddate': '2025-06-04T13:01:45-04:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) kpathsea version 6.3.4', 'subject': 'As directed by the National Artificial Intelligence Initiative Act of 2020 (P.L. 116-283), the goal of the AI RMF is to offer a resource to the organizations designing, developing, deploying, or using AI systems to help manage the many risks of AI and promote trustworthy and responsible development and use of AI systems. The Framework is intended to be voluntary, rights-preserving, non-sector specific, and use-case agnostic, providing flexibility to organizations of all sizes and in all sectors and throughout society to implement th

## 3) Chunking (텍스트 분할)

- `RecursiveCharacterTextSplitter`로 적당한 크기와 overlap 설정
- 파라미터를 바꾸며 결과 chunk 수/품질 차이를 관찰해 보세요


In [7]:
from typing import List
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(docs: List[Document], chunk_size=800, chunk_overlap=120) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    return splitter.split_documents(docs)

chunks = split_documents(docs, chunk_size=800, chunk_overlap=120)
print("Chunks:", len(chunks))
if chunks:
    print(chunks[0].metadata)
    print(chunks[0].page_content[:300])


Chunks: 541
{'producer': 'pdfTeX-1.40.24', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-01-24T14:45:46-05:00', 'author': 'National Institute of Standards and Technology', 'keywords': 'Artificial Intelligence (AI); AI; AI RMF; AI RMF 1.0; AI systems; trustworthy and responsible AI.', 'moddate': '2025-06-04T13:01:45-04:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) kpathsea version 6.3.4', 'subject': 'As directed by the National Artificial Intelligence Initiative Act of 2020 (P.L. 116-283), the goal of the AI RMF is to offer a resource to the organizations designing, developing, deploying, or using AI systems to help manage the many risks of AI and promote trustworthy and responsible development and use of AI systems. The Framework is intended to be voluntary, rights-preserving, non-sector specific, and use-case agnostic, providing flexibility to organizations of all sizes and in all sectors and throughout society to implement the appr

## 4) Embedding + Chroma 인덱싱 (persist)

- OpenAI Embeddings로 문서 조각을 벡터화
- Chroma 로컬 디렉토리에 저장하여 후속 세션에서 재사용 가능


In [8]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

def build_chroma_index(chunks: List[Document], db_dir: Path, collection_name="nist-security-docs"):
    embeddings = OpenAIEmbeddings()
    vectordb = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=str(db_dir),
        collection_name=collection_name,
    )
    vectordb.persist()
    return vectordb

vectordb = build_chroma_index(chunks, CHROMA_DIR, collection_name="nist-security-docs")
print("Chroma persisted at:", CHROMA_DIR)


Chroma persisted at: /Users/sjcha/Documents/6. 개인작업 관리(smilechacha)/@모두의연구소/@모두의연구소 - 04 - GC케어&유비케어/202511_gccare/gccare-langchain-rag-ai/chroma_store


/var/folders/jc/4z1x4d2x08v3pks_cvns5kzr0000gn/T/ipykernel_73352/595306487.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


## 5) Retriever 생성

- `vectordb.as_retriever(k=4)` 형태로 사용
- `k` 값은 다양하게 시도해 보세요 (예: 2, 4, 6)


In [9]:
from langchain_openai import OpenAIEmbeddings  # 재확인 용
from langchain_community.vectorstores import Chroma  # 재확인 용

def get_retriever(db_dir: Path, collection_name="nist-security-docs", k=4):
    embeddings = OpenAIEmbeddings()
    vectordb = Chroma(
        persist_directory=str(db_dir),
        embedding_function=embeddings,
        collection_name=collection_name,
    )
    return vectordb.as_retriever(search_kwargs={"k": k})

retriever = get_retriever(CHROMA_DIR, k=4)
print("Retriever ready.")


Retriever ready.


/var/folders/jc/4z1x4d2x08v3pks_cvns5kzr0000gn/T/ipykernel_73352/237766089.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectordb = Chroma(


## 6) LCEL 기반 RAG 체인 구축 (`question` → `answer`)

- `RunnableParallel`로 `{"context": retriever, "question": passthrough}` 구성
- `ChatPromptTemplate`에 Context/Question 주입
- `ChatOpenAI` 호출 → 최종 답변


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """
당신은 NIST 보안/AI 프레임워크 문서를 기반으로 답변하는 전문가입니다.
반드시 아래 Context 안의 내용만 사용하여 한국어로 답변하세요.
문서에 없는 내용이면 솔직하게 모른다고 말하세요.

# Context
{context}

# Question
{question}
""".strip()
)

llm = ChatOpenAI(model="gpt-4o-mini")

def join_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    RunnableParallel({
        "context": retriever | (lambda docs: join_docs(docs)),
        "question": RunnablePassthrough(),
    })
    | prompt
    | llm
)

print("RAG chain ready.")

## 7) 테스트 질의

아래 예시 질문으로 호출해 보고, LangSmith에서 Trace가 잘 찍히는지 확인해 봅시다.

- `"Zero Trust Architecture의 핵심 원칙을 3가지로 정리해줘."`
- `"NIST AI 리스크 관리 프레임워크의 주요 단계는?"`
- `"사내 정책 문서에 적용 가능한 베스트 프랙티스가 있나?"`


In [ ]:
questions = [
    "Zero Trust Architecture의 핵심 원칙을 3가지로 정리해줘.",
    "NIST AI 리스크 관리 프레임워크의 주요 단계는?",
    "문서에 없는 내용이면 모른다고 말해.",
]

for q in questions:
    print("\n=== Question ===\n", q)
    ans = rag_chain.invoke(q)
    try:
        print("\n--- Answer ---\n", ans.content)
    except AttributeError:
        print("\n--- Answer ---\n", ans)


## 8) 🧪 자유 실험 칸 (TODO)

아래 셀에 마음껏 실험해 보세요. 아이디어:

- `chunk_size`, `chunk_overlap`, `k` 값 바꿔보기
- Prompt에 "근거 문장/페이지 표시" 요구 추가해 보기
- 질문 스타일(정의/비교/절차/요약/표현 변경) 바꾸기
- `retriever` 대신 `vectordb.similarity_search` 직접 사용해 보기


In [ ]:
# TODO: 자유 실험 셀 1


In [ ]:
# TODO: 자유 실험 셀 2


## 9) (선택) 초기화/정리

- 실험을 반복하고 싶다면, `chroma_store` 폴더를 지우고 다시 인덱싱해 보세요.


In [ ]:
# 초기화 유틸 (실행 시 주의)
# import shutil
# shutil.rmtree(CHROMA_DIR, ignore_errors=True)
# CHROMA_DIR.mkdir(exist_ok=True)
# print("Chroma store cleared.")


## 10) LangSmith Trace 확인 팁

- 프로젝트: `gccare-rag-workshop`
- Run 하나를 열어 보면:
  - `retriever` 단계에서 어떤 chunk들이 선택되었는지
  - `prompt` 탭에서 Context/Question이 어떻게 주입됐는지
  - `llm` 응답과 토큰/지연시간 등을 확인할 수 있습니다.

> **권장 활동:** 안 좋았던 답변의 Run을 골라, 선택된 chunk/Prompt 지시를 점검하고 `k` 값/Prompt를 조정해 보세요.
